# End-to-End ML Producer Notebook
This notebook creates and operates on feature views. Producers manage feature views and entities in the feature store, facilitating collaboration and feature reuse across departments. This approach breaks down silos between training and serving environments and simplifies aggregations and moving window calculations using Python commands. Import the Producer Notebook into your Snowflake account by following the instructions provided below. This will create all the necessary feature views for the consumers.

## Setup
Import the importlib.metadata module and utilize it to fetch and print metadata for a specified list of packages.

In [ ]:
import importlib.metadata
package_names = ['snowbooks']#,'mlruntimes_service', 'mlruntimes_client']
for package_name in package_names:
	metadata = importlib.metadata.metadata(package_name)
	print(f"Metadata for {package_name}:")
	for key, value in metadata.items():
	    print(f"{key}: {value}")

Import the necessary packages for development.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from snowflake.snowpark import Window
from snowflake.snowpark.functions import *

from snowflake.snowpark.context import get_active_session
session = get_active_session()


from snowflake.snowpark import functions as F
from snowflake.ml.feature_store import (
FeatureStore,
FeatureView,
Entity,
CreationMode
)
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions

db = str(session.get_current_database().strip('"'))
solution_prefix = (db.upper()).split('_PROD')[0]


## Feature Store Schema & Privileges
Create a feature store schema and grant all necessary privileges for the producer to create feature store views, as well as the privileges for consumers to access these views.

In [ ]:
USE ROLE ACCOUNTADMIN;
SET FS_ROLE_PRODUCER = 'SYSADMIN';
SET FS_ROLE_CONSUMER = '{{solution_prefix}}_DATA_SCIENTIST';
SET FS_DATABASE = '{{solution_prefix}}_PROD';
SET FS_SCHEMA = 'FS_SCHEMA';
SET SCHEMA_FQN = CONCAT($FS_DATABASE, '.', $FS_SCHEMA);
SET FS_WAREHOUSE = '{{solution_prefix}}_DS_WH';
SET MR_DEMO_DB='{{solution_prefix}}_PROD';

-- Create schema
CREATE SCHEMA IF NOT EXISTS IDENTIFIER($FS_SCHEMA);

-- Build role hierarchy
GRANT ROLE IDENTIFIER($FS_ROLE_CONSUMER) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);

-- Grant PRODUCER role privileges
GRANT USAGE ON DATABASE IDENTIFIER($FS_DATABASE) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT USAGE ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT CREATE DYNAMIC TABLE ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT CREATE VIEW ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT CREATE TAG ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT CREATE DATASET ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);
GRANT APPLY TAG ON ACCOUNT TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);

-- Grant CONSUMER role privileges
GRANT USAGE ON DATABASE IDENTIFIER($FS_DATABASE) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT USAGE ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT SELECT,MONITOR ON FUTURE DYNAMIC TABLES IN SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT SELECT,MONITOR ON ALL DYNAMIC TABLES IN SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT SELECT,REFERENCES ON FUTURE VIEWS IN SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT SELECT,REFERENCES ON ALL VIEWS IN SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT CREATE DATASET ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);

-- [Optional] Grant USAGE ON WAREHOUSE to CONSUMER
GRANT USAGE ON WAREHOUSE IDENTIFIER($FS_WAREHOUSE) TO ROLE IDENTIFIER($FS_ROLE_CONSUMER);
GRANT USAGE ON SCHEMA IDENTIFIER($SCHEMA_FQN) TO ROLE IDENTIFIER($FS_ROLE_PRODUCER);


## Feature Store
Define the Feature Store

In [ ]:
session.sql('USE ROLE SYSADMIN')
session.sql(f'USE WAREHOUSE {solution_prefix}_DS_WH').collect()
session.sql('USE SCHEMA FS_SCHEMA').collect()

FS=FeatureStore(
session=session,
database=f"{solution_prefix}_PROD",
    name="FS_SCHEMA",
    default_warehouse=f"{solution_prefix}_DS_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST)

In [ ]:
menu_spdf = session.sql("SELECT menu_type, truck_brand_name, menu_item_name, item_category, item_subcategory, sale_price_usd FROM raw_pos.menu WHERE item_category != 'Beverage'");
menu_spdf

In [ ]:
cust_spdf = session.sql("SELECT customer_id, city, country, gender, marital_status, birthday_date, DATEDIFF(year, birthday_date, CURRENT_DATE()) AS age FROM raw_customer.customer_loyalty");
st.dataframe(cust_spdf)

In [ ]:
avg_monthly_purchase_amount = session.sql(f"SELECT  customer_id, ROUND(SUM(order_total) / (TIMESTAMPDIFF(MONTH, MIN(date), MAX(date)) + 1),2) AS avg_monthly_purchase_amount FROM {solution_prefix}_PROD.ANALYTICS.ORDERS_V GROUP BY customer_id")
avg_monthly_purchase_amount

In [ ]:
avg_weekly_purchase_amount = session.sql(f"SELECT customer_id,ROUND(SUM(order_total) / (TIMESTAMPDIFF(WEEK, MIN(date), MAX(date)) + 1),2) AS avg_weekly_purchase_amount FROM {solution_prefix}_PROD.ANALYTICS.ORDERS_V GROUP BY customer_id");
avg_weekly_purchase_amount

In [ ]:
avg_yearly_purchase_amount = session.sql(f"SELECT customer_id, ROUND(SUM(order_total)/(TIMESTAMPDIFF(YEAR, MIN(date), MAX(date)) + 1),2) AS avg_yearly_purchase_amount FROM {solution_prefix}_PROD.ANALYTICS.ORDERS_V GROUP BY customer_id")
avg_yearly_purchase_amount

In [ ]:
cust_avgs_spdf= avg_monthly_purchase_amount.join(avg_weekly_purchase_amount,"CUSTOMER_ID").join(avg_yearly_purchase_amount,"CUSTOMER_ID")
st.dataframe(cust_avgs_spdf)

Create feature Store Entities

In [ ]:

# Snowflake Feature Store requires an "entity" with "join_keys" be registered
custentity = Entity(name="CustomerIds", join_keys=["Customer_ID"])
FS.register_entity(custentity)

Menuentity = Entity(name="Menu_ItemNames", join_keys=["MENU_ITEM_NAME"])
FS.register_entity(Menuentity)


Purchaseavgs_entity = Entity(name="Purchase_Avgs", join_keys=["Customer_ID"])
FS.register_entity(Purchaseavgs_entity)

In [ ]:
FS.list_entities().show(100)

Register Customer Features View

In [ ]:
fv = FeatureView(
    name="CUSTOMER_FEATURES",
    entities = [custentity],
    feature_df=cust_spdf,
    refresh_freq="1 day"
)
registered_fv = FS.register_feature_view(
    feature_view=fv,
    version="V1",
     block=True,
    overwrite=True
)

Register Menu Features View

In [ ]:
fv = FeatureView(
    name="MENU_FEATURES",
    entities = [Menuentity],
    feature_df=menu_spdf,
    refresh_freq="1 day"
)

registered_fv = FS.register_feature_view(
    feature_view=fv,
    version="V1",
    block=True,
    overwrite=True
)

Register Purchase Features View

In [ ]:
fv = FeatureView(
    name="PURCHASE_FEATURES",
    entities = [Purchaseavgs_entity],
    feature_df=cust_avgs_spdf,
    refresh_freq="1 day"
)

registered_fv = FS.register_feature_view(
    feature_view=fv,
    version="V1",
     block=True,
    overwrite=True
)

In [ ]:
FS.list_feature_views(entity_name="CustomerIds")